In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC ##  Refined Layer - Analytical Metrics and Insights

# COMMAND ----------

from pyspark.sql.functions import col, countDistinct, avg, explode, lit
from pyspark.sql.window import Window

# Load Trusted data
df_trusted = spark.read.parquet("/FileStore/trusted/events_joined")

# COMMAND ----------

#  Most effective marketing channel (by offer completion rate)

# Filter completed and received events
completed = df_trusted.filter(col("event") == "offer completed")
received = df_trusted.filter(col("event") == "offer received")

# Explode channels for accurate grouping
completed_exploded = completed.withColumn("channel", explode(col("channels")))
received_exploded = received.withColumn("channel", explode(col("channels")))

# Group and count unique customers per channel
completed_channel = completed_exploded.groupBy("channel").agg(countDistinct("customer_id").alias("completed"))
received_channel = received_exploded.groupBy("channel").agg(countDistinct("customer_id").alias("received"))

# Join and calculate completion rate
channel_stats = completed_channel.join(received_channel, "channel") \
    .withColumn("completion_rate", (col("completed") / col("received")) * 100)

display(channel_stats.orderBy(col("completion_rate").desc()))


---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-7911428906803174>, line 26
     23 received_exploded = received.withColumn("channel", explode(col("channels")))
     25 # Group and count unique customers per channel
---> 26 completed_channel = completed_exploded.groupBy("channel").agg(countDistinct("customer_id").alias("completed"))
     27 received_channel = received_exploded.groupBy("channel").agg(countDistinct("customer_id").alias("received"))
     29 # Join and calculate completion rate

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:536, in DataFrame.groupBy(self, *cols)
    534     _cols.append(c)
    535 elif isinstance(c, str):
--> 536     _cols.append(self[c])
    537 elif isinstance(c, int) and not isinstance(c, bool):
    538     if c < 1:

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe

In [0]:
# Age distribution: offer completed vs not completed

# Customers who completed
completed_age = completed.select("customer_id", "age").dropDuplicates().withColumn("status", lit("completed"))

# Customers who received but did not complete
received_ids = received.select("customer_id", "offer_id").distinct()
completed_ids = completed.select("customer_id", "offer_id").distinct()

not_completed = received_ids.subtract(completed_ids)
not_completed_age = not_completed.join(df_trusted.select("customer_id", "age"), "customer_id").dropDuplicates() \
    .withColumn("status", lit("not completed"))

# Combine both for comparison
age_comparison = completed_age.unionByName(not_completed_age)

display(age_comparison.groupBy("status", "age").count().orderBy("age"))


In [0]:
#  Average time to complete an offer after receiving it

# Filter and rename for join
received_time = df_trusted.filter(col("event") == "offer received") \
    .select("customer_id", "offer_id", col("time").alias("start_time"))

completed_time = df_trusted.filter(col("event") == "offer completed") \
    .select("customer_id", "offer_id", col("time").alias("end_time"))

# Join and calculate duration
time_df = received_time.join(completed_time, ["customer_id", "offer_id"])
time_df = time_df.withColumn("completion_time", col("end_time") - col("start_time"))

# Average
avg_time = time_df.select(avg("completion_time").alias("avg_completion_time_hours"))
display(avg_time)


---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-7911428906803177>, line 16
     14 # Average
     15 avg_time = time_df.select(avg("completion_time").alias("avg_completion_time_hours"))
---> 16 display(avg_time)

File /databricks/python_shell/lib/dbruntime/display.py:142, in Display.display(self, input, *args, **kwargs)
    140 # This version is for Serverless + Spark Connect dogfooding.
    141 elif self.spark_connect_enabled and isinstance(input, ConnectDataFrame):
--> 142     self.display_connect_table(input, **kwargs)
    143 elif isinstance(input, ConnectDataFrame):
    144     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:103, in Display.display_connect_table(self, df, **kwargs)
     98 except Exception as e:
     99     raise type(
    100         e
    101     )("IPython shell encountered an error or was missing data, please resta